# # Feature Selection

Imports and Visualization parameters

In [ ]:
import pandas as pd
import numpy as np
import os
print os.getcwd();

# import libraries for visualization
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab

#import libraries for features decomposition
import sklearn as sk
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA

#Visualization param
pd.options.display.max_columns = None
pd.options.display.max_rows = None

Actually importing the data

In [3]:
#import data
df = pd.read_csv('../../Data/NBA-data-With-Positions.csv')
print "(rows,cols) =",df.shape
df.head(5)

(rows,cols) = (476, 30)


,Unnamed: 0,Player,TEAM,AGE,GP,W,L,MIN,FGM,FGA,FG%,03:00:00 PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,PF,DD2,TD3,PTS,+/-,PosicAbbrev
0,0,Aaron Brooks,CHI,31,69,36,33,16.1,2.7,6.8,40.1,1.0,2.7,35.7,0.7,0.9,76.6,0.3,1.2,1.5,2.6,1.2,0.4,0.1,1.9,0,0,7.1,-0.1,PG
1,1,Aaron Gordon,ORL,20,78,32,46,23.9,3.5,7.4,47.3,0.5,1.8,29.6,1.7,2.5,66.8,2.0,4.5,6.5,1.6,0.8,0.8,0.7,2.0,12,0,9.2,-1.0,PF
2,2,Aaron Harrison,CHA,21,21,15,6,4.4,0.2,0.9,26.3,0.1,0.5,30.0,0.2,0.6,41.7,0.2,0.5,0.7,0.1,0.2,0.3,0.0,0.5,0,0,0.9,-0.3,SG
3,3,Adreian Payne,MIN,25,52,18,34,9.3,1.0,2.8,36.6,0.2,0.6,28.1,0.3,0.5,65.4,0.4,1.8,2.1,0.6,0.7,0.3,0.2,1.5,0,0,2.5,-2.3,PF
4,4,Al Horford,ATL,30,82,48,34,32.1,6.5,12.8,50.5,1.1,3.1,34.4,1.3,1.6,79.8,1.8,5.5,7.3,3.2,1.3,0.8,1.5,2.0,17,0,15.2,2.8,C


In [4]:
dict = {' PG': 1, ' PF': 2, ' SG' : 3, ' SF' : 4, ' C' : 5}
invdict = {v: k for k, v in dict.iteritems()}

position_labels = [dict[x] for x in df['PosicAbbrev']]
df_numVar = df.select_dtypes(include=['number']).drop('Unnamed: 0',1)
df_numVar.head()

,AGE,GP,W,L,MIN,FGM,FGA,FG%,03:00:00 PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,PF,DD2,TD3,PTS,+/-
0,31,69,36,33,16.1,2.7,6.8,40.1,1.0,2.7,35.7,0.7,0.9,76.6,0.3,1.2,1.5,2.6,1.2,0.4,0.1,1.9,0,0,7.1,-0.1
1,20,78,32,46,23.9,3.5,7.4,47.3,0.5,1.8,29.6,1.7,2.5,66.8,2.0,4.5,6.5,1.6,0.8,0.8,0.7,2.0,12,0,9.2,-1.0
2,21,21,15,6,4.4,0.2,0.9,26.3,0.1,0.5,30.0,0.2,0.6,41.7,0.2,0.5,0.7,0.1,0.2,0.3,0.0,0.5,0,0,0.9,-0.3
3,25,52,18,34,9.3,1.0,2.8,36.6,0.2,0.6,28.1,0.3,0.5,65.4,0.4,1.8,2.1,0.6,0.7,0.3,0.2,1.5,0,0,2.5,-2.3
4,30,82,48,34,32.1,6.5,12.8,50.5,1.1,3.1,34.4,1.3,1.6,79.8,1.8,5.5,7.3,3.2,1.3,0.8,1.5,2.0,17,0,15.2,2.8


Feature Selection Methods

VarianceThreshold (this method will be used before all other ones)

In [51]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold()
newdf = sel.fit_transform(df_numVar)

1 - Laplacian Score

In [58]:
X = newdf

from skfeature.utility import construct_W
kwargs_W = {"metric":"euclidean","neighbor_mode":"knn","weight_mode":"heat_kernel","k":5,'t':1}
W = construct_W.construct_W(X, **kwargs_W)

from skfeature.function.similarity_based import lap_score
score = lap_score.lap_score(X, W=W)
print score

idx = lap_score.feature_ranking(score)
print idx

num_fea = 7
selected_features = df_numVar.columns[idx[0:num_fea]]
print selected_features

[  6.80346846e-09   9.93275462e-11   1.23519528e-10   4.62514027e-10
   3.87541221e-10   2.16717755e-11   3.84765220e-10   7.13824555e-11
   3.45057316e-13   2.81966672e-11   4.92272889e-12   8.01997579e-10
   1.12866172e-09   1.53157598e-10   5.09693088e-09   3.60272256e-09
   4.92164354e-09   7.00100644e-10   4.99454478e-09   1.23914258e-08
   6.77652390e-09   1.51168902e-08   9.40741929e-11   0.00000000e+00
   6.11444229e-12   2.86645672e-08]
[23  8 10 24  5  9  7 22  1  2 13  6  4  3 17 11 12 15 16 18 14 20  0 19 21
 25]
Index([u'TD3', u'03:00:00 PM', u'3P%', u'PTS', u'FGM', u'3PA', u'FG%'], dtype='object')


1 - Evaluating lap_score (esse usa um tipo de clusterização pra calcular essas paradas na real, acho que ele assume o kmeans)

In [59]:
from skfeature.utility import unsupervised_evaluation
import numpy as np
num_cluster = len(np.unique(position_labels))
#nmi,acc=unsupervised_evaluation.evaluation(X_selected=selected_features,n_clusters=num_cluster,y=position_labels)
#print nmi
#print acc

2 - SPEC

In [60]:
X = newdf

from skfeature.utility import construct_W
kwargs_W = {"metric":"euclidean","neighbor_mode":"knn","weight_mode":"heat_kernel","k":5,'t':1}
W = construct_W.construct_W(X, **kwargs_W)

from skfeature.function.similarity_based import SPEC
score = SPEC.spec(X, W=W)
print score

idx = SPEC.feature_ranking(score)
print idx

num_fea = 7
selected_features = df_numVar.columns[idx[0:num_fea]]
print selected_features

[  6.80347008e-09   9.93283191e-11   1.23519740e-10   4.62514014e-10
   3.87540474e-10   2.16719732e-11   3.84764976e-10   7.13789078e-11
   3.44926540e-13   2.81967299e-11   4.92250195e-12   8.01997781e-10
   1.12866183e-09   1.53158327e-10   5.09693126e-09   3.60272288e-09
   4.92164364e-09   7.00100629e-10   4.99454499e-09   1.23914254e-08
   6.77652414e-09   1.51168893e-08   9.40741989e-11  -6.67061507e-26
   6.11464989e-12   2.86645669e-08]
[25 21 19  0 20 14 18 16 15 12 11 17  3  4  6 13  2  1 22  7  9  5 24 10  8
 23]
Index([u'+/-', u'PF', u'STL', u'AGE', u'BLK', u'OREB', u'TOV'], dtype='object')


In [61]:
3 - MCFS

TypeError: unsupported operand type(s) for -: 'int' and 'module'

In [62]:
X = newdf

n_features = 7
from skfeature.utility import construct_W
kwargs_W = {"metric":"euclidean","neighbor_mode":"knn","weight_mode":"heat_kernel","k":5,'t':1}
W = construct_W.construct_W(X, **kwargs_W)

from skfeature.function.sparse_learning_based import MCFS
features_weight = MCFS.mcfs(X, n_features, W=W)
score = MCFS.feature_ranking(features_weight)
print score

num_fea = 7
selected_features = df_numVar.columns[score[0:num_fea]]
print selected_features

[ 8 17  9 12 22  0 21  7 24 23  1  2  3  4  5  6 19 18 10 11 13 14 15 16 20
 25]
Index([u'03:00:00 PM', u'AST', u'3PA', u'FTA', u'DD2', u'AGE', u'PF'], dtype='object')
